Top 10 patients with highest total claim amount

In [0]:
SELECT 
    pt.first_name || ' ' || pt.last_name AS patient_name,
    pt.patient_id, 
    pt.insurance_provider,
    pt.known_conditions,
    FLOOR(DATEDIFF(day, pt.date_of_birth::date, CURRENT_DATE) / 365.25) AS age,
    COUNT(c.claim_id) AS total_claims,
    SUM(c.claim_amount) AS total_billed,
    SUM(c.paid_amount) AS total_paid,
    SUM(c.claim_amount) - SUM(c.paid_amount) AS unpaid_amount,
    ROUND((SUM(c.paid_amount)::numeric / NULLIF(SUM(c.claim_amount), 0) * 100), 2) AS payment_rate_pct
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.patients_data pt ON c.patient_id = pt.patient_id
GROUP BY pt.patient_id, pt.first_name, pt.last_name, pt.insurance_provider, pt.known_conditions, pt.date_of_birth
ORDER BY total_billed DESC
LIMIT 10;

Patients with >3 emergency visits in last 6 months

In [0]:
SELECT 
    pt.first_name || ' ' || pt.last_name AS patient_name,
    pt.insurance_provider,
    pt.known_conditions,
    FLOOR(DATEDIFF(day, pt.date_of_birth::date, CURRENT_DATE) / 365.25) AS age,
    COUNT(c.claim_id) AS emergency_visits_6m,
    SUM(c.claim_amount) AS total_emergency_billed,
    SUM(c.paid_amount) AS total_emergency_paid,
    MIN(c.service_start_date) AS first_emergency,
    MAX(c.service_start_date) AS last_emergency
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.patients_data pt ON c.patient_id = pt.patient_id
WHERE c.is_emergency = true
  AND c.service_start_date >= DATEADD(month, -6, CURRENT_DATE)
GROUP BY pt.patient_id, pt.first_name, pt.last_name, pt.insurance_provider, pt.known_conditions, pt.date_of_birth
HAVING COUNT(c.claim_id) > 3
ORDER BY emergency_visits_6m DESC;

Age-wise distribution of claim frequency

In [0]:
SELECT 
    age_group,
    COUNT(DISTINCT patient_id) AS unique_patients,
    COUNT(claim_id) AS total_claims,
    ROUND(AVG(claim_amount), 2) AS avg_claim_amount,
    SUM(CASE WHEN is_emergency = true THEN 1 ELSE 0 END) AS emergency_claims,
    ROUND((COUNT(claim_id) * 1.0 / COUNT(DISTINCT patient_id)), 2) AS claims_per_patient
FROM (
    SELECT 
        c.claim_id,
        c.claim_amount,
        c.is_emergency,
        c.service_start_date,
        pt.patient_id,
        CASE 
            WHEN FLOOR(DATEDIFF(day, pt.date_of_birth::date, c.service_start_date::date) / 365.25) BETWEEN 0 AND 17 THEN '0-17 (Pediatric)'
            WHEN FLOOR(DATEDIFF(day, pt.date_of_birth::date, c.service_start_date::date) / 365.25) BETWEEN 18 AND 34 THEN '18-34 (Young Adult)'
            WHEN FLOOR(DATEDIFF(day, pt.date_of_birth::date, c.service_start_date::date) / 365.25) BETWEEN 35 AND 49 THEN '35-49 (Adult)'
            WHEN FLOOR(DATEDIFF(day, pt.date_of_birth::date, c.service_start_date::date) / 365.25) BETWEEN 50 AND 64 THEN '50-64 (Middle Age)'
            ELSE '65+ (Senior)'
        END AS age_group
    FROM healthcare_cp.hcp.claims_data c
    JOIN healthcare_cp.hcp.patients_data pt ON c.patient_id = pt.patient_id
) age_claims
GROUP BY age_group
ORDER BY 
    CASE age_group
        WHEN '0-17 (Pediatric)' THEN 1
        WHEN '18-34 (Young Adult)' THEN 2
        WHEN '35-49 (Adult)' THEN 3
        WHEN '50-64 (Middle Age)' THEN 4
        WHEN '65+ (Senior)' THEN 5
    END;

Patients with large difference between billed and paid amounts

In [0]:
SELECT 
    pt.first_name || ' ' || pt.last_name AS patient_name,
    pt.insurance_provider,
    pt.known_conditions,
    COUNT(c.claim_id) AS total_claims,
    SUM(c.claim_amount) AS total_billed,
    SUM(c.paid_amount) AS total_paid,
    SUM(c.claim_amount) - SUM(c.paid_amount) AS unpaid_amount,
    ROUND(((SUM(c.claim_amount) - SUM(c.paid_amount)) / SUM(c.claim_amount) * 100), 2) AS unpaid_percentage,
    COUNT(CASE WHEN c.claim_status = 'Denied' THEN 1 END) AS denied_claims
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.patients_data pt ON c.patient_id = pt.patient_id
GROUP BY pt.patient_id, pt.first_name, pt.last_name, pt.insurance_provider, pt.known_conditions
HAVING SUM(c.claim_amount) > 10000
   AND ((SUM(c.claim_amount) - SUM(c.paid_amount)) / SUM(c.claim_amount)) > 0.5
ORDER BY unpaid_amount DESC
LIMIT 15;

Avg claims per patient by insurance provider

In [0]:
SELECT 
    pt.insurance_provider,
    COUNT(DISTINCT pt.patient_id) AS total_patients,
    COUNT(c.claim_id) AS total_claims,
    ROUND((COUNT(c.claim_id) * 1.0 / COUNT(DISTINCT pt.patient_id)), 2) AS avg_claims_per_patient,
    ROUND(AVG(c.claim_amount), 2) AS avg_claim_amount,
    SUM(c.claim_amount) AS total_billed,
    SUM(c.paid_amount) AS total_paid,
    ROUND((SUM(c.paid_amount) / SUM(c.claim_amount) * 100), 2) AS payment_rate_pct
FROM healthcare_cp.hcp.patients_data pt
LEFT JOIN healthcare_cp.hcp.claims_data c ON pt.patient_id = c.patient_id
GROUP BY pt.insurance_provider
ORDER BY avg_claims_per_patient DESC;

Identify high-risk patients (frequent emergency + high unpaid bills)

In [0]:
WITH patient_risk_metrics AS (
    SELECT 
        pt.patient_id,
        pt.first_name || ' ' || pt.last_name AS patient_name,
        pt.insurance_provider,
        pt.known_conditions,
        FLOOR(DATEDIFF(day, pt.date_of_birth::date, CURRENT_DATE) / 365.25) AS age,
        COUNT(c.claim_id) AS total_claims,
        SUM(CASE WHEN c.is_emergency = true THEN 1 ELSE 0 END) AS emergency_claims,
        SUM(c.claim_amount) AS total_billed,
        SUM(c.paid_amount) AS total_paid,
        SUM(c.claim_amount) - SUM(c.paid_amount) AS unpaid_amount,
        COUNT(CASE WHEN c.service_start_date >= DATEADD(month, -6, CURRENT_DATE) AND c.is_emergency = true THEN 1 END) AS recent_emergencies
    FROM healthcare_cp.hcp.patients_data pt
    JOIN healthcare_cp.hcp.claims_data c ON pt.patient_id = c.patient_id
    GROUP BY pt.patient_id, pt.first_name, pt.last_name, pt.insurance_provider, pt.known_conditions, pt.date_of_birth
)
SELECT 
    patient_name,
    insurance_provider,
    known_conditions,
    age,
    total_claims,
    emergency_claims,
    recent_emergencies,
    total_billed,
    total_paid,
    unpaid_amount,
    ROUND((unpaid_amount / total_billed * 100), 2) AS unpaid_percentage,
    CASE 
        WHEN recent_emergencies >= 3 AND unpaid_amount > 50000 THEN 'CRITICAL RISK'
        WHEN recent_emergencies >= 2 AND unpaid_amount > 25000 THEN 'HIGH RISK'
        WHEN recent_emergencies >= 1 AND unpaid_amount > 15000 THEN 'MODERATE RISK'
        ELSE 'LOW RISK'
    END AS risk_category
FROM patient_risk_metrics
WHERE (recent_emergencies >= 2 OR unpaid_amount > 15000)
ORDER BY 
    CASE 
        WHEN recent_emergencies >= 3 AND unpaid_amount > 50000 THEN 1
        WHEN recent_emergencies >= 2 AND unpaid_amount > 25000 THEN 2
        WHEN recent_emergencies >= 1 AND unpaid_amount > 15000 THEN 3
        ELSE 4
    END,
    unpaid_amount DESC;